##### 问题:
我们正在编写需要使用回调函数的代码（比如，事件处理例程、完成回调等），但是希
望回调函数可以携带额外的状态以便在回调函数内部使用。

##### 解决方案:
本节中提到的对回调函数的应用可以在许多库和框架中找到——尤其是那些和异步处
理相关的库和框架。为了说明和测试的目的，我们首先定义下面的函数，它会调用一
个回调函数：

In [56]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    # Invoke the callback with the result
    callback(result)

def print_result(result):
    print('Got:', result)

def add(x, y):
    return x + y

# apply_async(add, (2, 3), callback=print_result)

在现实世界中，类似这样的代码可能会完成各种高级的处理任务，这会涉及线程、进
程和定时器等，但我们这里主要关注的不是这些。相反，我们只是把注意力集中在对
回调函数的调用上。下面的示例展示了上述代码应该如何使用：

In [57]:
def print_result(result):
    print('Got:', result)

def add(x, y):
    return x + y

apply_async(add, (2, 3), callback=print_result)

Got: 5


我们会注意到函数 print_result()仅接受一个单独的参数，也就是 result。这里并没有传
入其他的信息到函数中。有时候当我们希望回调函数可以同其他变量或者部分环境进
行交互时，缺乏这类信息就会带来问题。

一种在回调函数中携带额外信息的方法是使用绑定方法（bound-method）而不是普通
的函数。比如，下面这个类保存了一个内部的序列号码，每当接收到一个结果时就递
增这个号码。

In [58]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))

要使用这个类，可以创建一个类实例并将绑定方法 handler 当做回调函数来用：


In [59]:
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)
apply_async(add, ('hello', 'world'), callback=r.handler)

[1] Got: 5
[2] Got: helloworld


作为类的替代方案，也可以使用闭包来捕获状态。示例如下：

In [60]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence   # nonlocal 声明用来表示变量 sequence 是在回调函数中修改的
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler 

In [61]:
handler = make_handler()
apply_async(add, (2, 3), callback=handler)
apply_async(add, ('hello', 'world'), callback=handler)

[1] Got: 5
[2] Got: helloworld


除此之外还有一种方法，有时候可以利用协程（coroutine）来完成同样的任务：

In [62]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))


对于协程来说，可以使用它的 send()方法来作为回调函数，就像下面这样：

In [63]:
handler = make_handler()
next(handler) # Advance to the yield
apply_async(add, (2, 3), callback=handler.send)

[1] Got: 5


最后但也同样重要的是，也可以通过额外的参数在回调函数中携带状态，然后用 partial()
来处理参数个数的问题（见 7.8 节）。示例如下：

In [64]:
class SequenceNo:
    def __init__(self):
        self.sequence = 0

def handler(result, seq):
    seq.sequence += 1
    print('[{}] Got: {}'.format(seq.sequence, result))
seq = SequenceNo()
from functools import partial

In [65]:
apply_async(add, (2, 3), callback=partial(handler, seq=seq))
apply_async(add, ('hello', 'world'), callback=partial(handler, seq=seq))

[1] Got: 5
[2] Got: helloworld


基于回调函数的软件设计常常会面临使代码陷入一团乱麻的风险。部分原因是因为
从代码发起初始请求开始到回调执行的这个过程中，回调函数常常是与这个环境相
脱离的。因此，在发起请求和处理结果之间的执行环境就丢失了。如果想让回调函
数在涉及多个步骤的任务处理中能够继续执行，就必须清楚应该如何保存和还原相
关的状态。

主要有两种方法可用于捕获和携带状态。可以在类实例上携带状态（将状态附加到绑
定方法上），也可以在闭包中携带状态。这两种方法中，闭包可能要显得更轻量级一些，
而且由于闭包也是由函数构建的，这样显得会更加自然。这两种方法都可以自动捕获
所有正在使用的变量。因此，这就使得我们不必担心哪个具体的状态需要保存起来（根
据代码自动决定哪些需要保存）。

如果使用闭包，那么需要对可变变量多加留意。在给出的解决方案中，nonlocal 声明用来
表示变量 sequence 是在回调函数中修改的。没有这个声明，将得到错误提示。

将协程用作回调函数的有趣之处在于这种方式和采用闭包的方案关系紧密。从某种意
义上说，协程甚至更为清晰，因为这里只出现了一个单独的函数。此外，变量都可以
自由地进行修改，不必担心 nonlocal 声明。可能存在的缺点在于人们对协程的理解程
度不如其他的 Python 特性。使用协程时还有几个小技巧需要掌握，比如在使用协程前
需要先对其调用一次 next()，这在实践中常常容易忘记。不过，协程还有其他的潜在用
途，比如定义内联的回调函数（在下一节中讲解）。

如果所有需要做的就是在回调函数中传入额外的值，那么最后提到的那个有关 partial()
的技术是很管用的。有时候我们也会看到用 lambda 表达式来实现同样的功能：

In [66]:
apply_async(add, (2, 3), callback=lambda r: handler(r, seq))

[3] Got: 5


要查看更多的示例请参见 7.8 节。在那一节中我们展示了如何利用 partial()来修改函数
的参数签名。